<a href="https://colab.research.google.com/github/kkrusere/youTube-comments-Analyzer/blob/main/YT_comments_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,189 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,393 kB]
Fetched 3,839 kB in 4s (1,079 kB/s)
Reading package lists... Done
Buildin

In [2]:
import os
from google.colab import drive

In [3]:
#mounting google drive
drive.mount('/content/drive')

########################################

#changing the working directory
os.chdir("/content/drive/MyDrive/EV NLP Data")

!pwd


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/EV NLP Data


In [4]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import random
import time


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=chrome_options)


from bs4 import BeautifulSoup
import requests
import json

In [5]:
import pandas as pd
import numpy as np


In [23]:
import operator
from google.colab import userdata
api_key = userdata.get('YouTubeAPI_key')

In [24]:
import googleapiclient.discovery
from googleapiclient.discovery import build
import datetime



youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)


This Jupyter Notebook is designed to collect YouTube video comments from various videos. These comments will be used for creating, training, and validating a Sentiment Analysis model. The videos from which the comments were collected were chosen with no particular criteria other than being from my favorite channels and videos.

The process of collecting comments makes use of the YouTube API, Selenium, BeautifulSoup, and other custom functions.

The process:

1. Create a list of favorite channels.
2. Use the YouTube API to select the top 10 most-watched videos from each channel and store them in a master list of videos from which we will collect comments.
3. For each video in the master video list, use Selenium and BeautifulSoup to collect the comments and store them in a pandas DataFrame.
4. Clean and sanitize the comments in the DataFrame and prepare the data for Sentiment Analysis.

In [8]:
# below are functions for reading a writting json file for the current working directory

def save_to_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)

def load_from_json(filename):
    with open(filename, 'r') as json_file:
        comments = json.load(json_file)
    return comments

In [9]:
# # We are going to save this of channels to a json file. There was no particular proceedure in creating the below list of channels other than trying to capture cannels that would create a balance on positive, negative and neutral comments.

# # List of YouTube channel URLs
# channels = [
#     "https://www.youtube.com/@BestEverFoodReviewShow",
#     "https://www.youtube.com/@TheRamseyShow",
#     "https://www.youtube.com/@MrBallen",
#     "https://www.youtube.com/@MattWalsh",
#     "https://www.youtube.com/@VICE",
#     "https://www.youtube.com/@CNN",
#     "https://www.youtube.com/@Jeep",
#     "https://www.youtube.com/@MrBeast",
#     "https://www.youtube.com/@obamawhitehouse",
#     "https://www.youtube.com/@DonaldJTrumpforPresident",
#     "https://www.youtube.com/@TheDailyShow",
#     "https://www.youtube.com/@gordonramsay",
#     "https://www.youtube.com/@frontline",
#     "https://www.youtube.com/@JoeBiden",
#     "https://www.youtube.com/@LastWeekTonight",
#     "https://www.youtube.com/@edmundscars",
#     "https://www.youtube.com/@BillBurrOfficial",
#     "https://www.youtube.com/@netflixisajoke",
#     "https://www.youtube.com/@TheLaughFactory",
#     "https://www.youtube.com/@VICENews",
#     "https://www.youtube.com/@SargonofAkkad",
#     "https://www.youtube.com/@gags",
#     "https://www.youtube.com/@LaurenSouthernOfficial",
#     "https://www.youtube.com/@FunnyOrDie",
#     "https://www.youtube.com/@dropout",
#     "https://www.youtube.com/@thedavidpakmanshow",
#     "https://www.youtube.com/@Vaush",
#     "https://www.youtube.com/@ComedyCentral",
#     "https://www.youtube.com/@SortedFood",
#     "https://www.youtube.com/@gmc",
#     "https://www.youtube.com/@mytruecrimenews",
#     "https://www.youtube.com/@amazon",
#     "https://www.youtube.com/@RealCandaceO",
#     "https://www.youtube.com/@JCS",
#     "https://www.youtube.com/@Google",
#     "https://www.youtube.com/@ChrisHansen",
#     "https://www.youtube.com/@PragerU",
#     "https://www.youtube.com/@NBCSports",
#     "https://www.youtube.com/@Apple",
#     "https://www.youtube.com/@ExploreWithUs",
#     "https://www.youtube.com/@TODAY",
#     "https://www.youtube.com/@Microsoft",
#     "https://www.youtube.com/@DatelineNBC",
#     "https://www.youtube.com/@Timcast",
#     "https://www.youtube.com/@AssociatedPress",
#     "https://www.youtube.com/@JordanBPeterson",
#     "https://www.youtube.com/@NBCNews",
#     "https://www.youtube.com/@StevenCrowder",
#     "https://www.youtube.com/@NFL",
#     "https://www.youtube.com/@SabineHossenfelder",
#     "https://www.youtube.com/@RubinReport",
#     "https://www.youtube.com/@CNBC",
#     "https://www.youtube.com/@DailyWirePlus",
#     "https://www.youtube.com/@joerogan",
#     "https://www.youtube.com/@60minutes",
#     "https://www.youtube.com/@ElectricVehicleSpace",
#     "https://www.youtube.com/@DougDeMuro",
#     "https://www.youtube.com/@markets",
#     "https://www.youtube.com/@nypost",
#     "https://www.youtube.com/@neoexplains",
#     "https://www.youtube.com/@unboxtherapy",
#     "https://www.youtube.com/@KitchenNightmares",
#     "https://www.youtube.com/@SamEckholm",
#     "https://www.youtube.com/@RealStories",
#     "https://www.youtube.com/@FreeDocumentary",
#     "https://www.youtube.com/@TechQuarks",
#     "https://www.youtube.com/@WonderDocs",
#     "https://www.youtube.com/@sciencechannel",
#     "https://www.youtube.com/@FluctusOfficial",
#     "https://www.youtube.com/@Munchies",
#     "https://www.youtube.com/@dcofficial",
#     "https://www.youtube.com/@48hours",
#     "https://www.youtube.com/@therealautoblog",
#     "https://www.youtube.com/@BusinessInsider",
#     "https://www.youtube.com/@NickDiGiovanni",
#     "https://www.youtube.com/@RamTrucks",
#     "https://www.youtube.com/@stalekrackerofficial4240",
#     "https://www.youtube.com/@nytimes",
#     "https://www.youtube.com/@DailyCallerVideo",
#     "https://www.youtube.com/@babishculinaryuniverse",
#     "https://www.youtube.com/@marvel",
#     "https://www.youtube.com/@adidas",
#     "https://www.youtube.com/@nissanusa",
#     "https://www.youtube.com/@TED",
#     "https://www.youtube.com/@CBSNews",
#     "https://www.youtube.com/@ProHomeCooks",
#     "https://www.youtube.com/@buzzfeedtasty",
#     "https://www.youtube.com/@Honda",
#     "https://www.youtube.com/@TFLEV",
#     "https://www.youtube.com/@jimgaffigan",
#     "https://www.youtube.com/@breakingpoints",
#     "https://www.youtube.com/@Honest_Ads",
#     "https://www.youtube.com/@Shaun_vids",
#     "https://www.youtube.com/@JKenjiLopezAlt",
#     "https://www.youtube.com/@epicurious",
#     "https://www.youtube.com/@JoshuaWeissman",
#     "https://www.youtube.com/@TheOnion",
#     "https://www.youtube.com/@Chevrolet",
#     "https://www.youtube.com/@HowItShouldHaveEnded",
#     "https://www.youtube.com/@wsj",
#     "https://www.youtube.com/@cracked",
#     "https://www.youtube.com/@TheTRYChannel",
#     "https://www.youtube.com/@TeamCoco",
#     "https://www.youtube.com/@LOLNetwork",
#     "https://www.youtube.com/@KeyAndPeele",
#     "https://www.youtube.com/@bonappetit",
#     "https://www.youtube.com/@MentourNow",
#     "https://www.youtube.com/@HasanAbi",
#     "https://www.youtube.com/@TheLateLateShow",
#     "https://www.youtube.com/@ThatChapter",
#     "https://www.youtube.com/@Coffeezilla",
#     "https://www.youtube.com/@JimmyKimmelLive",
#     "https://www.youtube.com/@MentourPilot",
#     "https://www.youtube.com/@OpenAI",
#     "https://www.youtube.com/@TheFBIFiles",
#     "https://www.youtube.com/@CoffeehouseCrime",
#     "https://www.youtube.com/@TechLead",
#     "https://www.youtube.com/@gustoonz",
#     "https://www.youtube.com/@TFLcar",
#     "https://www.youtube.com/@BreakfastClubPower1051FM",
#     "https://www.youtube.com/@nike",
#     "https://www.youtube.com/@MaydayAirDisaster",
#     "https://www.youtube.com/@GingerBilly",
#     "https://www.youtube.com/@TheBabylonBee",
#     "https://www.youtube.com/@Donut",
#     "https://www.youtube.com/@CSPAN",
#     "https://www.youtube.com/@PhilosophyTube",
#     "https://www.youtube.com/@RealTime",
#     "https://www.youtube.com/@kbb",
#     "https://www.youtube.com/@tesla",
#     "https://www.youtube.com/@TheYoungTurks",
#     "https://www.youtube.com/@bigthink",
#     "https://www.youtube.com/@LindsayEllisVids",
#     "https://www.youtube.com/@latimes",
#     "https://www.youtube.com/@SecularTalk",
#     "https://www.youtube.com/@hbomberguy",
#     "https://www.youtube.com/@NewsNation",
#     "https://www.youtube.com/@aljazeeraenglish",
#     "https://www.youtube.com/@BenShapiro",
#     "https://www.youtube.com/@BadFaithPodcast",
#     "https://www.youtube.com/@fifa",
#     "https://www.youtube.com/@espn",
#     "https://www.youtube.com/@ActualJusticeWarrior",
#     "https://www.youtube.com/@ContraPoints",
#     "https://www.youtube.com/@fallontonight",
#     "https://www.youtube.com/@MarkWiens",
#     "https://www.youtube.com/@BBC",
#     "https://www.youtube.com/@pbsspacetime",
#     "https://www.youtube.com/@Dodge",
#     "https://www.youtube.com/@InsideEdition",
#     "https://www.youtube.com/@ford",
#     "https://www.youtube.com/@FoxNews",
#     "https://www.youtube.com/@Beardmeatsfood",
#     "https://www.youtube.com/@CodeBlueCam",
#     "https://www.youtube.com/@SamChui",
#     "https://www.youtube.com/@toyotausa",
#     "https://www.youtube.com/@GugaFoods",
#     "https://www.youtube.com/@BBCNews",
#     "https://www.youtube.com/@ColbertLateShow",
#     "https://www.youtube.com/@thehill",
#     "https://www.youtube.com/@NatGeo",
#     "https://www.youtube.com/@Cadillac",
#     "https://www.youtube.com/@TheDodo",
#     "https://www.youtube.com/@discovery",
#     "https://www.youtube.com/@SimplilearnOfficial",
#     "https://www.youtube.com/@NBCNews"
# ]

# # Data to be written to JSON file
# data = {"channels": channels}

# # Writing to json file
# save_to_json(data, "channels.json")

In [10]:
#

In [11]:
# Reading the channel list from the saved jjson file
json_data = load_from_json("channels.json")
channel_list = json_data['channels']

In [12]:
## Method 1 - Using the YouTube API

# Function to get the channel ID from a URL that has `channel handles`
def get_channel_id_from_url_YTAPI(api_key, custom_url):
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Removing the 'https://www.youtube.com/' part from the URL
    username = custom_url.split('/')[-1]

    # Using the 'search' method to find the channel
    request = youtube.search().list(
        part='snippet',
        q=username,
        type='channel',
        maxResults=1
    )

    response = request.execute()

    if 'items' in response and len(response['items']) > 0:
        channel_id = response['items'][0]['snippet']['channelId']
        return channel_id
    else:
        print(f"ChannelID not found for URL: {custom_url}")
        return None

In [27]:
# Function to get all video IDs from the channel
def get_all_video_ids(channel_id):
    video_ids = []
    next_page_token = None

    while True:
        request = youtube.search().list(
            part='id',
            channelId=channel_id,
            maxResults=50,
            pageToken=next_page_token,
            type='video'
        )
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['id']['videoId'])

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return video_ids

# Function to get video statistics (including view count)
def get_video_stats(video_ids):
    video_stats = []
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part='statistics',
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()
        if response.status_code == 200:
          # Process the successful response here.
          print("Request successful!")
        else:
          # Handle the error response here.
          print("Error:", response.status_code)

        for item in response['items']:
            view_count = int(item['statistics']['viewCount'])
            video_stats.append((item['id'], view_count))

    return video_stats

In [26]:
# prompt: if response was successfull

if response.status_code == 200:
  # Process the successful response here.
  print("Request successful!")
else:
  # Handle the error response here.
  print("Error:", response.status_code)


NameError: name 'response' is not defined

In [13]:
## Method 2 - Using Request and BeautifulSoup

# Function to get the channel ID from a URL that has `channel handles`
def get_channel_id_from_url2(url):

  # Send a request to fetch the HTML content of the page
  response = requests.get(url)

  # Parse the HTML content using BeautifulSoup
  soup = BeautifulSoup(response.text, 'html.parser')

  # Find the channel ID from the HTML content
  # The channel ID is usually within a link tag with the attribute containing '/channel/'
  channel_id = None
  for link in soup.find_all('link'):
      href = link.get('href', '')
      if '/channel/' in href:
          channel_id = href.split('/channel/')[1]
          break

  return channel_id


In [14]:
# We are going to create a dictionary that we will turn into a data frame containing
  # 1. Channel Name
  # 2. Channel Url (with handle)
  # 3. Channel ID
  # 4. Chennel UrlwID (with ID)
channel_dict = {
    "channel_name": [],
    "channel_url": [],
    "channel_id": [],
    "chennel_UrlwID": []
}


In [15]:
# Function to return th channel `username`
def get_channel_username(url):
  return url.split('@')[-1]

In [16]:
# let's populate our data dictionary using the channel list
random.shuffle(channel_list) # randomly shuffle the channel list
for channel_url in channel_list:
  channel_dict["channel_name"].append(get_channel_username(channel_url))
  channel_dict["channel_url"].append(channel_url)
  time.sleep(2) #wait for 2 seconds
  channel_id = get_channel_id_from_url2(channel_url)
  channel_dict["channel_id"].append(channel_id)
  channel_dict["chennel_UrlwID"].append(f'https://www.youtube.com/channel/{channel_id}')


In [17]:
# Now we convert the `channel_dict` to a pandas dataframe
df = pd.DataFrame(channel_dict)
df.head()

,channel_name,channel_url,channel_id,chennel_UrlwID
0,Google,https://www.youtube.com/@Google,UCK8sQmJBp8GCxrOtXWBpyEA,https://www.youtube.com/channel/UCK8sQmJBp8GCx...
1,60minutes,https://www.youtube.com/@60minutes,UCsN32BtMd0IoByjJRNF12cw,https://www.youtube.com/channel/UCsN32BtMd0IoB...
2,DougDeMuro,https://www.youtube.com/@DougDeMuro,UCsqjHFMB_JYTaEnf_vmTNqg,https://www.youtube.com/channel/UCsqjHFMB_JYTa...
3,NBCSports,https://www.youtube.com/@NBCSports,UCqZQlzSHbVJrwrn5XvzrzcA,https://www.youtube.com/channel/UCqZQlzSHbVJrw...
4,NatGeo,https://www.youtube.com/@NatGeo,UCpVm7bg6pXKo1Pr6k5kxG9A,https://www.youtube.com/channel/UCpVm7bg6pXKo1...


In [28]:
for CHANNEL_ID in df['channel_id']:
  print(CHANNEL_ID)

  # Get all video IDs from the channel
  video_ids = get_all_video_ids(CHANNEL_ID)

  # Get statistics for all videos
  video_stats = get_video_stats(video_ids)

  # Sort videos by view count in descending order
  sorted_videos = sorted(video_stats, key=operator.itemgetter(1), reverse=True)

  # Get the top 10 most viewed videos
  top_10_videos = sorted_videos[:10]

  # Print the top 10 videos
  for i, (video_id, view_count) in enumerate(top_10_videos, 1):
      print(f"{i}. Video ID: {video_id} - {view_count} views")


UCK8sQmJBp8GCxrOtXWBpyEA


AttributeError: 'dict' object has no attribute 'status_code'

In [ ]:
# prompt: request error code

if response.status_code == 200:
  # Process the successful response here.
  print("Request successful!")
else:
  # Handle the error response here.
  print("Error:", response.status_code)


In [42]:
# URL of the YouTube channel's videos page
url = "https://www.youtube.com/channel/UCkLXELm63_pH7L-r-548kig/videos"

# Send a request to fetch the page content
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Function to parse the views and title
def get_views_and_title(video):
    title = video.find('a', id='video-title').text.strip()
    views = video.find('span', class_='style-scope ytd-grid-video-renderer').text.strip()
    return title, views

# Get the list of videos
videos = soup.find_all('ytd-grid-video-renderer', class_='style-scope ytd-grid-renderer')

# Extract title and views for each video
video_data = [get_views_and_title(video) for video in videos]

# Sort videos by views (convert views to integers for comparison)
def extract_views(view_string):
    return int(view_string.split(' ')[0].replace(',', ''))

video_data.sort(key=lambda x: extract_views(x[1]), reverse=True)

# Get the top 10 most viewed videos
top_10_videos = video_data[:10]

for i, (title, views) in enumerate(top_10_videos, start=1):
    print(f"{i}. {title} - {views}")


In [45]:
# URL of the YouTube channel's videos page
url = "https://www.youtube.com/channel/UCkLXELm63_pH7L-r-548kig/videos"

# Open the YouTube channel's videos page
driver.get(url)
time.sleep(3)  # Wait for the page to load

# Scroll down to load more videos
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    # Wait to load the page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Get video titles and view counts
videos = driver.find_elements(By.ID, 'dismissible')
video_data = []

for video in videos:
    title = video.find_element(By.ID, 'video-title').text
    views_text = video.find_element(By.XPATH, './/div[@id="metadata-line"]/span[1]').text
    if "views" in views_text:
        views = int(views_text.split()[0].replace(",", ""))
        video_data.append((title, views))

# Sort videos by view count
video_data.sort(key=lambda x: x[1], reverse=True)

# Get top 10 most viewed videos
top_10_videos = video_data[:10]

# Print the top 10 videos
for i, (title, views) in enumerate(top_10_videos, 1):
    print(f"{i}. {title} - {views} views")


In [47]:
videos

[<selenium.webdriver.remote.webelement.WebElement (session="59ed7f1d8287c6c6cd722e5584a9bef4", element="f.D094E65B3261BFAD265CB11F9C6EE998.d.C0157FFA3C8B27E6EC5BD3EF6ADFDBFE.e.28")>,
 <selenium.webdriver.remote.webelement.WebElement (session="59ed7f1d8287c6c6cd722e5584a9bef4", element="f.D094E65B3261BFAD265CB11F9C6EE998.d.C0157FFA3C8B27E6EC5BD3EF6ADFDBFE.e.34")>,
 <selenium.webdriver.remote.webelement.WebElement (session="59ed7f1d8287c6c6cd722e5584a9bef4", element="f.D094E65B3261BFAD265CB11F9C6EE998.d.C0157FFA3C8B27E6EC5BD3EF6ADFDBFE.e.85")>,
 <selenium.webdriver.remote.webelement.WebElement (session="59ed7f1d8287c6c6cd722e5584a9bef4", element="f.D094E65B3261BFAD265CB11F9C6EE998.d.C0157FFA3C8B27E6EC5BD3EF6ADFDBFE.e.86")>,
 <selenium.webdriver.remote.webelement.WebElement (session="59ed7f1d8287c6c6cd722e5584a9bef4", element="f.D094E65B3261BFAD265CB11F9C6EE998.d.C0157FFA3C8B27E6EC5BD3EF6ADFDBFE.e.87")>,
 <selenium.webdriver.remote.webelement.WebElement (session="59ed7f1d8287c6c6cd722e558

In [ ]:
def get_channel_videos(channel_url):
    # Extract channel ID from URL
    if 'channel/' in channel_url:
        channel_id = channel_url.split('channel/')[-1]
    else:
        # Handle other URL formats if necessary
        raise ValueError('Unsupported URL format. Please provide a valid channel URL.')


    # Get channel uploads playlist ID
    response = youtube.channels().list(
        part='contentDetails',
        id=channel_id
    ).execute()

    uploads_playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    # Get all videos in the uploads playlist
    videos = []
    next_page_token = None
    while True:
        response = youtube.playlistItems().list(
            part='contentDetails',
            playlistId=uploads_playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        videos += response['items']
        next_page_token = response.get('nextPageToken')

        if next_page_token is None:
            break

    return [video['contentDetails']['videoId'] for video in videos]

In [ ]:
def get_video_comments_count(video_id, youtube):
    response = youtube.videos().list(
        part='statistics',
        id=video_id
    ).execute()

    return int(response['items'][0]['statistics'].get('commentCount', 0))

In [ ]:
def get_top_5_videos(channel_url):
    video_ids = get_channel_videos(channel_url)

    youtube = build('youtube', 'v3', developerKey=API_KEY)

    video_comments = []
    for video_id in video_ids:
        comment_count = get_video_comments_count(video_id, youtube)
        video_comments.append((video_id, comment_count))

    # Sort videos by comment count in descending order and get top 5
    top_5_videos = sorted(video_comments, key=lambda x: x[1], reverse=True)[:5]

    return top_5_videos




In [ ]:
# Example usage
channel_url = 'https://www.youtube.com/channel/CHANNEL_ID'
top_videos = get_top_5_videos(channel_url)
print(top_videos)

In [ ]:
channel_id = 'UCfuEu0ff9Yy_oA5U21C5Stw'

request = youtube.search().list(
    part='snippet',
    channelId=channel_id,
    type='video',
    order='viewCount',
    maxResults=10
)
video_url_list = []

response = request.execute()

for item in response['items']:
    video_title = item['snippet']['title']
    video_id = item['id']['videoId']
    video_url = f'https://www.youtube.com/watch?v={video_id}'

    print(f'Title: {video_title} | URL: {video_url}')


Title: I killed the QUEEN to save the hive. | URL: https://www.youtube.com/watch?v=DXuzQkXVKXI
Title: What comes out of an egg this BIG? The 50 day hatching adventure. | URL: https://www.youtube.com/watch?v=Cp19XhbqJ_c
Title: The Round Beehive Explained... | URL: https://www.youtube.com/watch?v=qcjj9PjhBdU
Title: We tricked our hens into hatching out the RAREST baby peacocks! | URL: https://www.youtube.com/watch?v=8rwC8FQugrw
Title: What’s INSIDE of our Emu Eggs? | URL: https://www.youtube.com/watch?v=vIqhcG1oy2w
Title: A New Bee Emerges | URL: https://www.youtube.com/watch?v=3JsX0OC58d4
Title: These are NOT your normal big, green eggs. | URL: https://www.youtube.com/watch?v=jjEAZrZbHnU
Title: How many Emu Eggs are there? | URL: https://www.youtube.com/watch?v=m9a2LYfgjkA
Title: Meet the Guicken. How did this happen? | URL: https://www.youtube.com/watch?v=LY5lxc2lMjE
Title: Our Emus Finally Laid Eggs *after 4 years* | URL: https://www.youtube.com/watch?v=J8fTVQpWsCQ


In [ ]:
def get_channel_id_from_handle_url(handle_url):
    """Fetches the YouTube channel ID from a URL containing only the handle.

    This function uses the YouTube Data API's 'search' endpoint to look for
    channels matching the given handle. It extracts the channel ID from the
    first matching result.

    Args:
        handle_url: The YouTube channel URL containing only the handle (e.g.,
            'https://www.youtube.com/@Google').

    Returns:
        The channel ID (string) if found, otherwise None.

    Raises:
        googleapiclient.errors.HttpError: If there's an issue with the API request.
    """
    try:
        request = youtube.search().list(
            part='snippet',
            q=handle_url,   # Search for the handle
            type='channel',
            maxResults=1    # We only need one result
        )
        response = request.execute()

        if response['items']:
            channel_id = response['items'][0]['snippet']['channelId']
            return channel_id

    except googleapiclient.errors.HttpError as e:
        print(f"An error occurred: {e}")

    return None  # Return None if no channel is found


In [ ]:
from urllib.parse import urlparse, parse_qs

def get_channel_id_from_handle(handle_url):
    """Extracts the YouTube channel ID from a URL containing only the handle.

    This function takes a YouTube channel URL that might be in various formats
    (shortened, custom, etc.) and extracts the unique channel ID. It works even
    if the URL only contains the handle, utilizing the YouTube Data API for the lookup.

    Args:
        handle_url: The YouTube channel URL (string) containing the handle.

    Returns:
        The channel ID (string) if found, or None if not found or on API error.

    Raises:
        googleapiclient.errors.HttpError: If there's an issue with the API request.
    """

    try:
        # Try direct extraction from URL if it's a standard format
        parsed_url = urlparse(handle_url)
        if parsed_url.netloc == 'www.youtube.com' and parsed_url.path.startswith('/c/'):
            return parsed_url.path.split('/')[2]  # Handle after /c/
        elif parsed_url.netloc == 'www.youtube.com' and parsed_url.path.startswith('/channel/'):
            return parsed_url.path.split('/')[2]
        else:
            query_params = parse_qs(parsed_url.query)
            if 'ab_channel' in query_params:
                return query_params['ab_channel'][0]  # from query parameter
    except ValueError:
        # Parsing failed, continue with API lookup
        pass

    # If direct extraction fails, use the YouTube Data API for lookup
    request = youtube.search().list(
        part='snippet',
        q=handle_url,  # Search by the handle
        type='channel',
        maxResults=1
    )

    response = request.execute()

    if response['items']:
        return response['items'][0]['snippet']['channelId']  # Extract ID from result

    return None  # Channel not found or error occurred


In [ ]:
get_channel_id_from_handle_url('https://www.youtube.com/@thehill')

'UCfuEu0ff9Yy_oA5U21C5Stw'

In [ ]:
get_channel_id_from_handle('https://www.youtube.com/@thehill')


'UCfuEu0ff9Yy_oA5U21C5Stw'

In [ ]:
UCPWXiRWZ29zrxPFIQT7eHSA

In [ ]:
def get_video_list(channel_id):
  """Fetches a list of top-viewed video URLs from a YouTube channel.

  This function uses the YouTube Data API to retrieve the details (title, ID)
  of the most viewed videos on a specified channel. It constructs full URLs for
  each video and returns them in a list.

  Args:
      channel_id: The YouTube channel ID (string) from which to retrieve videos.

  Returns:
      A list of strings representing the URLs of the top-viewed videos.
  """

  request = youtube.search().list(
    part='snippet',
    channelId=channel_id,
    type='video',
    order='viewCount',
    maxResults=10
  )
  video_url_list = []

  response = request.execute()

  for item in response['items']:
      video_title = item['snippet']['title']
      video_id = item['id']['videoId']
      video_url = f'https://www.youtube.com/watch?v={video_id}'

      video_url_list.append(video_url)


  return video_url_list

In [ ]:
# YouTube video URL
video_url = "https://www.youtube.com/watch?v=-cJ1fSX3Nec"

In [ ]:
try:
    wait = WebDriverWait(driver, 5)

    # Open the YouTube video
    driver.get(video_url)

    wait.until(EC.visibility_of_element_located((By.TAG_NAME, 'body')))

    # Scroll down to load comments
    last_height = 0
    while True:
        # Scroll to the end of the page
        #driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        # Scroll using page-manager element's scroll height
        driver.execute_script("window.scrollTo(0, document.getElementById('page-manager').scrollHeight);")
        time.sleep(5)  # Adjust sleep time as needed

        # Get the current page height
        new_height = driver.execute_script("return document.getElementById('page-manager').scrollHeight")

        # Break the loop if no more content is loaded
        if new_height == last_height:
            break

        last_height = new_height

    # Wait for the comments section to be visible
    comments_section = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "ytd-comments#comments.style-scope.ytd-watch-flexy")))

    # Get the HTML content of the comments section
    comments_html = comments_section.get_attribute('outerHTML')

finally:
    # Close the WebDriver session
    driver.quit()



In [ ]:
soup = BeautifulSoup(comments_html, 'html.parser')

# Find the span element with the specified class
comment_count_span = soup.find('span', class_='style-scope yt-formatted-string')

# Extract the text content of the span element
comment_count = comment_count_span.text.strip()

# Print or use the comment count
print("Comment Count:", comment_count)

Comment Count: 9,327


In [ ]:
# Find all occurrences of the ytd-comment-thread-renderer element
comment_thread_renderers = soup.find_all('ytd-comment-thread-renderer', class_='style-scope ytd-item-section-renderer')

# Count the number of occurrences
comment_thread_count = len(comment_thread_renderers)

# Print or use the comment thread count
print("Number of ytd-comment-thread-renderer elements:", comment_thread_count)

Number of ytd-comment-thread-renderer elements: 957


In [ ]:
# Function to save comments data to a JSON file
def save_comments_to_json(comments, filename = 'youtube_comments.json'):
    with open(filename, 'w') as json_file:
        json.dump(comments, json_file, indent=4)

def load_comments_from_json(filename = 'youtube_comments.json'):
    with open(filename, 'r') as json_file:
        comments = json.load(json_file)
    return comments

In [ ]:
comments = []
comments_data = load_comments_from_json()
# Iterate through each comment thread renderer
for comment_thread_renderer in comment_thread_renderers:

    # Extracting the comment text
    comment_text_element = comment_thread_renderer.find('yt-attributed-string', id='content-text')
    comment_text = comment_text_element.get_text(strip=True) if comment_text_element else None

    # Extracting the number of likes
    like_count_element = comment_thread_renderer.find('span', class_='style-scope ytd-comment-engagement-bar')
    like_count = like_count_element.get_text(strip=True) if like_count_element else None

    # Extracting the number of replies
    reply_count_element = comment_thread_renderer.find('ytd-button-renderer', id='more-replies')
    reply_count = reply_count_element.get_text(strip=True) if reply_count_element else None

    comments.append(comment_text)

    print("Comment Text:", comment_text)
    print("Like Count:", like_count)
    print("Reply Count:", reply_count)

    print("\n\n\n")


    comments_data.append(
        {
        "comment_text": comment_text,
        "like_count": like_count,
        "reply_count": reply_count

        }
    )


save_comments_to_json(comments = comments_data)

In [ ]:
len(load_comments_from_json())

16477

In [ ]:
def get_video_id(video_url):
  video_id = video_url.split('=')[1]

  return video_id

In [ ]:
# Request video details
response = youtube.videos().list(
    part="snippet,statistics",
    id=get_video_id(video_url)
).execute()

# Extract required information
video = response["items"][0]
title = video["snippet"]["title"]
thumbnail_url = video["snippet"]["thumbnails"]["maxres"]["url"]
view_count = video["statistics"]["viewCount"]
like_count = video["statistics"]["likeCount"]
commentCount = video["statistics"]["commentCount"]
date_posted = video["snippet"]["publishedAt"]
description = video["snippet"]["description"]


date_str = date_posted
date_object = datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")

# Format the date object
date_posted = date_object.strftime("%Y-%m-%d %H:%M:%S")


# Split the description by newlines
paragraphs = description.split("\n")

# Get the first paragraph
first_paragraph = paragraphs[0]

# Generate HTML to display the thumbnail
thumbnail_html = f'<img src="{thumbnail_url}" alt="Thumbnail">'

# Print the information
print("Title:", title)
print("Thumbnail URL:", thumbnail_html)
print("View count:", view_count)
print("Like count:", like_count)
print("Number of Comments:", commentCount)
print("Date posted:", date_posted)
print("Description:", first_paragraph)


Title: Why EVs Are Piling Up At Dealerships In The U.S.
Thumbnail URL: <img src="https://i.ytimg.com/vi/cZlsZwcIgpc/maxresdefault.jpg" alt="Thumbnail">
View count: 4492458
Like count: 36502
Number of Comments: 23211
Date posted: 2023-10-16 16:26:47
Description: In August 2023, it took about twice as long to sell an EV in the U.S. as it did the previous January. Prices of EVs are down 22% year-over-year and that's mainly driven by Tesla. About two thirds of EVs sold are Elon Musk's brand. Companies like Ford have ramped up hybrid production as demand has leveled off. While slightly more than half of consumers say EVs are the future and will eventually replace Internal Combustion Engines, less than a third of dealers say so. This all comes at a time when investments in EVs are more than ever. So what's really going on? Watch the video to learn more.


In [ ]:
# Search for videos using the YouTube API
def search_videos(query, youtube = youtube):

  # Request video search
  response = youtube.search().list(
      q=query,
      type="video",
      part="id,snippet",
      maxResults=10
  ).execute()

  # Extract video IDs from search results
  video_ids = []
  for item in response["items"]:
    video_ids.append(item["id"]["videoId"])

  return video_ids

In [ ]:
def get_video_info(video_id):
  # Request video details
  response = youtube.videos().list(
      part="snippet,statistics",
      id=video_id
  ).execute()

  # Extract required information
  video = response["items"][0]
  title = video["snippet"]["title"]
  view_count = video["statistics"]["viewCount"]
  like_count = video["statistics"]["likeCount"]
  commentCount = video["statistics"]["commentCount"]
  date_posted = video["snippet"]["publishedAt"]
  description = video["snippet"]["description"]


  date_str = date_posted
  date_object = datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")

  # Format the date object
  date_posted = date_object.strftime("%Y-%m-%d %H:%M:%S")


  # Split the description by newlines
  paragraphs = description.split("\n")

  # Get the first paragraph
  first_paragraph = paragraphs[0]

  extracted_video_url = f"https://www.youtube.com/watch?v={video_id}"


  # Print the information
  print("Title:", title)
  print("Video URL:", extracted_video_url)
  print("View count:", view_count)
  print("Like count:", like_count)
  print("Number of Comments:", commentCount)
  print("Date posted:", date_posted)
  print("Description:", first_paragraph)
  print("\n\n\n")


In [ ]:
search_query = "Electric Vehicles"
video_ids = search_videos(search_query)


for video_id in video_ids:
  get_video_info(video_id)

Title: Should I Get An Electric Car?
Video URL: https://www.youtube.com/watch?v=vXjs17ydRgI
View count: 40707
Like count: 2661
Number of Comments: 574
Date posted: 2024-04-10 15:31:46
Description: Check out Insectarium on PBS Terra: https://www.youtube.com/watch?v=jowk_A71-EU




Title: Used electric cars. The FIVE golden buying rules | Electrifying
Video URL: https://www.youtube.com/watch?v=aIYyycWQc34
View count: 13639
Like count: 682
Number of Comments: 96
Date posted: 2024-04-09 15:30:00
Description: #usedelectric #buyingcars #electriccars




Title: Are used EVs a rip-off?!
Video URL: https://www.youtube.com/watch?v=Ur47okU3eUk
View count: 350718
Like count: 13192
Number of Comments: 1927
Date posted: 2024-04-10 10:05:09
Description: Sell your car for free with Carwow: https://bit.ly/Sell-Your-Car-For-Free-1004




Title: Why Hybrids Are Beating EVs In The U.S.
Video URL: https://www.youtube.com/watch?v=R_J_T7TWbXY
View count: 633530
Like count: 9340
Number of Comments: 2979
Date 